# Treinamento dos principais comandos de SQL e SPARK

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## Create Table

In [0]:
%sql --indica que iremos utilizar um comando SQL na celula
CREATE OR REPLACE TABLE spark_catalog.bronze.tr_pessoa(
  id INT COMMENT 'Chave Primária',
  primeiro_nome VARCHAR(255) COMMENT 'Primeiro nome da pessoa',
  ultimo_nome VARCHAR(255) COMMENT 'Ultimo nome da pessoa',
  data_nascimento DATE COMMENT 'Data de nascimento'
)
LOCATION 'dbfs:/FileStore/landing_zone/tr_pessoa'
TBLPROPERTIES ('comment' = 'Esta tabela comtém dados de pessoas.')

In [0]:
%sql
describe bronze.tr_pessoa

col_name,data_type,comment
id,int,Chave Primária
primeiro_nome,varchar(255),Primeiro nome da pessoa
ultimo_nome,varchar(255),Ultimo nome da pessoa
data_nascimento,date,Data de nascimento


In [0]:
%sql
-- criando tabela rendimento anual pessoa
CREATE OR REPLACE TABLE spark_catalog.bronze.tr_pessoa_rendimento(
  id INT COMMENT 'Chave primaria',
  id_pessoa int COMMENT 'Id pessoa',
  rendimento_anual double comment 'rendimento anual',
  ano_rendimento int COMMENT 'ano rendimento'
)
LOCATION 'dbfs:/FileStore/landing_zone/tr_pessoa_rendimento'
TBLPROPERTIES ('comment' = 'Esta tabela contém dados de rendimento de pessoas')

## Visualizando o Schema da Tabela

In [0]:
%sql
describe spark_catalog.bronze.tr_pessoa

col_name,data_type,comment
id,int,Chave Primária
primeiro_nome,varchar(255),Primeiro nome da pessoa
ultimo_nome,varchar(255),Ultimo nome da pessoa
data_nascimento,date,Data de nascimento


In [0]:
%sql
describe spark_catalog.bronze.tr_pessoa_rendimento

col_name,data_type,comment
id,int,Chave primaria
id_pessoa,int,Id pessoa
rendimento_anual,double,rendimento anual
ano_rendimento,int,ano rendimento


## Realizando um insert na tabela

In [0]:
%sql
-- Inserindo dados na tabela de pessoas
insert into spark_catalog.bronze.tr_pessoa
  (id, primeiro_nome, ultimo_nome, data_nascimento)
values 
  (1, 'Anderson', 'Leal', '2003-01-15'),
  (2, 'Felix', 'Souza', '1998-01-20'), 
  (3, 'Junior', 'Amorim','1994-02-11'),
  (4, 'Barbara', 'Souza','2001-03-22'),
  (5, 'Jordam', 'Avilar','2001-03-22')

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
-- inserindo dados na tabela pessoas redimentos
insert into spark_catalog.bronze.tr_pessoa_rendimento
  (id, id_pessoa, rendimento_anual, ano_rendimento)
values
  (1, 1, 710235, 2022),
  (2, 1, 140450, 2023), 
  (3, 1, 100000, 2024),
  (4, 2, 214700, 2022),
  (5, 2, 510235, 2023),
  (6, 2, 340450, 2024), 
  (7, 3, 180000, 2022),
  (8, 3, 204000, 2023),
  (9, 3, 717235, 2024),
  (10, 4, 140490, 2022),
  (12, 4, 214000, 2024)

num_affected_rows,num_inserted_rows
11,11


## Lendo: Spark ou SQL

In [0]:
%sql
-- ler passando a location
SELECT *
FROM delta.`dbfs:/FileStore/landing_zone/tr_pessoa/`

id,primeiro_nome,ultimo_nome,data_nascimento
1,Anderson,Leal,2003-01-15
2,Felix,Souza,1998-01-20
3,Junior,Amorim,1994-02-11
4,Barbara,Souza,2001-03-22
5,Jordam,Avilar,2001-03-22


In [0]:
%sql
-- Lendo usando a tabela
SELECT *
FROM bronze.tr_pessoa

id,primeiro_nome,ultimo_nome,data_nascimento
1,Anderson,Leal,2003-01-15
2,Felix,Souza,1998-01-20
3,Junior,Amorim,1994-02-11
4,Barbara,Souza,2001-03-22
5,Jordam,Avilar,2001-03-22


In [0]:
%scala
val df = spark.read.format("delta").load("dbfs:/FileStore/landing_zone/tr_pessoa")
df.show()

+---+-------------+-----------+---------------+
 id|primeiro_nome|ultimo_nome|data_nascimento|
+---+-------------+-----------+---------------+
 1| Anderson| Leal| 2003-01-15|
 2| Felix| Souza| 1998-01-20|
 3| Junior| Amorim| 1994-02-11|
 4| Barbara| Souza| 2001-03-22|
 5| Jordam| Avilar| 2001-03-22|
+---+-------------+-----------+---------------+

df: org.apache.spark.sql.DataFrame = [id: int, primeiro_nome: string ... 2 more fields]

In [0]:
%python
spark.table('spark_catalog.bronze.tr_pessoa_rendimento').display()

id,id_pessoa,rendimento_anual,ano_rendimento
1,1,710235.0,2022
2,1,140450.0,2023
3,1,100000.0,2024
4,2,214700.0,2022
5,2,510235.0,2023
6,2,340450.0,2024
7,3,180000.0,2022
8,3,204000.0,2023
9,3,717235.0,2024
10,4,140490.0,2022


In [0]:
%python
df = spark.sql('select * from spark_catalog.bronze.tr_pessoa_rendimento')
df.display()

id,id_pessoa,rendimento_anual,ano_rendimento
1,1,710235.0,2022
2,1,140450.0,2023
3,1,100000.0,2024
4,2,214700.0,2022
5,2,510235.0,2023
6,2,340450.0,2024
7,3,180000.0,2022
8,3,204000.0,2023
9,3,717235.0,2024
10,4,140490.0,2022
